# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
csv_file = "../WeatherPy/output_data/city_data.csv"

df = pd.read_csv(csv_file)

df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.1203,-134.9692,79.20,73,97,10.67,PF,1644103722
1,ushuaia,-54.8000,-68.3000,53.26,58,20,29.93,AR,1644103813
2,albany,42.6001,-73.9662,15.10,57,5,1.01,US,1644103842
3,castro,-24.7911,-50.0119,65.25,97,79,1.92,BR,1644104043
4,kapaa,22.0752,-159.3190,80.19,64,0,3.00,US,1644103971


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key) 

# Convert Humidity to float and store
# Also, handle NaN values
df = df.dropna()
humidity = df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = df[["Lat", "Lng"]].astype(float)

In [17]:
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=400,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_df = df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_df = filtered_df.loc[(filtered_df["Max Temp"] < 80) & (filtered_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_df = filtered_df.loc[filtered_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_df = filtered_df.loc[filtered_df["Cloudiness"] == 0]

# Drop any rows with null values.
filtered_df = filtered_df.dropna()

filtered_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,sur,22.5667,59.5289,70.61,50,0,6.44,OM,1644104045
17,sao filipe,14.8961,-24.4956,72.97,73,0,9.42,CV,1644103828
26,cape town,-33.9258,18.4232,72.12,66,0,1.01,ZA,1644104007
249,la orilla,17.9833,-102.2333,79.23,60,0,6.49,MX,1644104114
251,nioro,13.3500,-15.7500,78.28,15,0,4.32,GM,1644104115
256,natitingou,10.3042,1.3796,73.74,15,0,8.03,BJ,1644104117
432,shasta lake,40.6804,-122.3708,74.12,18,0,6.71,US,1644103932
441,la palma,33.8464,-118.0467,77.41,19,0,5.01,US,1644103903
491,vicuna,-30.0319,-70.7081,79.77,40,0,4.38,CL,1644104186


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
hotel_df = filtered_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing closest hotel... skipping.")
        
    print("------------------")

Retrieving Results for Index 7: sur.
Closest hotel is Sur Plaza Hotel.
------------------
Retrieving Results for Index 17: sao filipe.
Closest hotel is Hotel Xaguate.
------------------
Retrieving Results for Index 26: cape town.
Closest hotel is Radisson Blu Hotel Waterfront, Cape Town.
------------------
Retrieving Results for Index 249: la orilla.
Closest hotel is Baymont by Wyndham Lazaro Cardenas.
------------------
Retrieving Results for Index 251: nioro.
Missing closest hotel... skipping.
------------------
Retrieving Results for Index 256: natitingou.
Closest hotel is Auberge Centrale.
------------------
Retrieving Results for Index 432: shasta lake.
Closest hotel is Shasta Dam Motel.
------------------
Retrieving Results for Index 441: la palma.
Closest hotel is La Quinta Inn & Suites by Wyndham Buena Park.
------------------
Retrieving Results for Index 491: vicuna.
Closest hotel is Los Mosaicos.
------------------


In [27]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,sur,22.5667,59.5289,70.61,50,0,6.44,OM,1644104045,Sur Plaza Hotel
17,sao filipe,14.8961,-24.4956,72.97,73,0,9.42,CV,1644103828,Hotel Xaguate
26,cape town,-33.9258,18.4232,72.12,66,0,1.01,ZA,1644104007,"Radisson Blu Hotel Waterfront, Cape Town"
249,la orilla,17.9833,-102.2333,79.23,60,0,6.49,MX,1644104114,Baymont by Wyndham Lazaro Cardenas
251,nioro,13.3500,-15.7500,78.28,15,0,4.32,GM,1644104115,NaN
256,natitingou,10.3042,1.3796,73.74,15,0,8.03,BJ,1644104117,Auberge Centrale
432,shasta lake,40.6804,-122.3708,74.12,18,0,6.71,US,1644103932,Shasta Dam Motel
441,la palma,33.8464,-118.0467,77.41,19,0,5.01,US,1644103903,La Quinta Inn & Suites by Wyndham Buena Park
491,vicuna,-30.0319,-70.7081,79.77,40,0,4.38,CL,1644104186,Los Mosaicos


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))